# Compose main modelling table for publication

Use the table resulting from `evaluate_session_level_ccc.py` (&rarr; `session_level_analysis/compiled-session_level-paper_configs.csv`) with the best performing model for each target and survey, with eGeMAPS and wav2vec2, and with noisy and denoised data respectively.  
Take that table and convert it to LaTeX source. In the process, change the raw, souce-like strings that e.g., describe the models, into nice and presentable labels. This table can readily be pasted into the LaTeX project of the publication.  

In [1]:
import os
import pandas as pd

# Define the paths
path_results_base = "../../results/mwas/composed/session_level_analysis"
table_paper = "compiled-session_level-paper_configs.csv"

# Construct the full path to the table
table_paper_path = os.path.join(path_results_base, table_paper)

# Read the table from the .ods file
df_paper = pd.read_csv(table_paper_path)
df_paper_raw = df_paper.copy()

In [2]:
# Insert column "Quality"

# Determine the position of the "Target" column
target_index = df_paper.columns.get_loc("Target")

# Insert the "Quality" column after the "Target" column
df_paper.insert(target_index + 1, "Quality", "")

# Fill the "Quality" column conditionally based on the "Survey" column
df_paper["Quality"] = df_paper["Survey"].apply(
    lambda x: "Noisy" if "noisy" in x else "Denoised"
)

In [3]:
df_paper

,ccc_conf_mean,ccc_conf_low,ccc_conf_high,lower_bound_larger_null,task,Features,ccc_segment_mean,ccc_segment_low,ccc_segment_high,concordance_cc-test-agg-average,path,Target,Quality,Task,Survey
0,0.363824,-0.044603,0.590668,False,speechtasks-nilago-happy,eGeMAPSv02,0.354083,0.027664,0.567943,0.363824,/who_5_percentage_score_corrected/raw_0_100-no...,who_5_percentage_score_corrected,Denoised,speechtasks-nilago-happy,outer_cv_loso_fixed-continuous-surveys_all
1,0.238113,-0.015007,0.452820,False,speechtasks-sustained_utterance-a,wav2vec2-variant-wav2vec2-large-robust-ft-libr...,0.348391,0.012134,0.539457,0.238113,/who_5_percentage_score_corrected/raw_0_100-no...,who_5_percentage_score_corrected,Denoised,speechtasks-sustained_utterance-a,outer_cv_loso_fixed-continuous-surveys_all
2,0.350369,0.019678,0.515335,True,speechtasks-nilago-happy,eGeMAPSv02,0.361475,0.031440,0.514476,0.350369,/who_5_percentage_score_corrected/raw_0_100-no...,who_5_percentage_score_corrected,Noisy,speechtasks-nilago-happy,outer_cv_loso_fixed-continuous-surveys_all-noisy
3,0.340419,0.088999,0.497717,True,speechtasks-nilago-neutral,wav2vec2-variant-wav2vec2-large-robust-ft-libr...,0.288002,0.063689,0.481959,0.340419,/who_5_percentage_score_corrected/raw_0_100-no...,who_5_percentage_score_corrected,Noisy,speechtasks-nilago-neutral,outer_cv_loso_fixed-continuous-surveys_all-noisy
4,0.114707,-0.184313,0.315105,False,speechtasks-spontaneous-work_tasks,eGeMAPSv02,0.082212,-0.099834,0.276044,0.114707,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,pss_10_total_score,Denoised,speechtasks-spontaneous-work_tasks,outer_cv_loso_fixed-continuous-surveys_all
5,0.172311,-0.122455,0.402716,False,speechtasks-nilago-neutral,wav2vec2-variant-wav2vec2-large-robust-ft-libr...,0.162778,-0.160369,0.336582,0.172311,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,pss_10_total_score,Denoised,speechtasks-nilago-neutral,outer_cv_loso_fixed-continuous-surveys_all
6,0.151166,-0.113080,0.360953,False,speechtasks-spontaneous-work_tasks,eGeMAPSv02,0.098678,-0.082959,0.261394,0.151166,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,pss_10_total_score,Noisy,speechtasks-spontaneous-work_tasks,outer_cv_loso_fixed-continuous-surveys_all-noisy
7,0.250923,0.058861,0.402575,True,speechtasks-spontaneous-work_tasks,wav2vec2-variant-wav2vec2-large-robust-12-ft-e...,0.117312,0.020579,0.236165,0.250923,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,pss_10_total_score,Noisy,speechtasks-spontaneous-work_tasks,outer_cv_loso_fixed-continuous-surveys_all-noisy
8,0.021402,-0.278219,0.277513,False,speechtasks-sustained_utterance-a,eGeMAPSv02,0.064470,-0.164220,0.318883,0.021402,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,phq_8_total_score,Denoised,speechtasks-sustained_utterance-a,outer_cv_loso_fixed-continuous-surveys_all
9,0.171450,-0.052023,0.345443,False,speechtasks-nilago-happy,wav2vec2-variant-wav2vec2-large-robust-ft-libr...,0.194951,-0.037036,0.383240,0.171450,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,phq_8_total_score,Denoised,speechtasks-nilago-happy,outer_cv_loso_fixed-continuous-surveys_all


In [4]:
# Extract the "Model" from the "path" column
df_paper["Model"] = df_paper["path"].str.extract(
    r"/([^/]+)/models/results-compiled\.yaml$"
)

# Replace strings in the "Model" column based on the specified conditions
df_paper["Model"] = df_paper["Model"].apply(
    lambda x: (
        "XGBr"
        if "XGBRegressor" in x
        else (
            "LR"
            if "LinearRegression" in x
            else (
                "SVR"
                if "SVR" in x
                else ("KNNr" if "KNeighborsRegressor-lower" in x else x)
            )
        )
    )
)

# Insert the "Model" column right after the "Task" column
quality_index = df_paper.columns.get_loc("Task")
df_paper.insert(quality_index + 1, "Model", df_paper.pop("Model"))

In [5]:
# Replace strings in the "Survey" column based on the specified conditions
df_paper["Survey"] = df_paper["Survey"].apply(
    lambda x: (
        "All"
        if "surveys_all" in x
        else ("Daily" if "daily" in x else ("Weekly" if "weekly" in x else x))
    )
)

In [6]:
# Replace strings in the "Target" column based on the specified conditions
df_paper["Target"] = df_paper["Target"].apply(
    lambda x: (
        "WHO-5"
        if "who_5_percentage_score_corrected" in x
        else (
            "PSS-10"
            if "pss_10_total_score" in x
            else (
                "Stress-now"
                if "stress_current" in x
                else (
                    "PHQ-8"
                    if "phq_8_total_score" in x
                    else ("Stress-work" if "stress_work_tasks" in x else x)
                )
            )
        )
    )
)

In [7]:
# Replace strings in the "Task" column based on the specified conditions
df_paper["Task"] = df_paper["Task"].apply(
    lambda x: (
        "Read-neutral"
        if "nilago-neutral" in x
        else (
            "Read-happy"
            if "speechtasks-nilago-happy" in x
            else (
                "All"
                if "speechtasks-standardized_tasks" in x
                else (
                    "Counting"
                    if "speechtasks-counting" in x
                    else (
                        "Spontaneous"
                        if "speechtasks-spontaneous-work_tasks" in x
                        else (
                            "Sustained /a/"
                            if "speechtasks-sustained_utterance-a" in x
                            else x
                        )
                    )
                )
            )
        )
    )
)

In [8]:
# Replace strings in the "Features" column based on the specified conditions
df_paper["Features"] = df_paper["Features"].apply(
    lambda x: (
        "W2V2-LR-LIBRI"
        if "wav2vec2-large-robust-ft-libri-960h-num_hidden_layers-0" in x
        else (
            "W2V2-LR-MSP"
            if "wav2vec2-large-robust-12-ft-emotion-msp-dim-num_hidden_layers-0" in x
            else (
                "W2V2-L-XLSR"
                if "wav2vec2-large-xlsr-53-num_hidden_layers-0" in x
                else ("eGeMAPS" if "eGeMAPSv02" in x else x)
            )
        )
    )
)

In [9]:
df_paper

,ccc_conf_mean,ccc_conf_low,ccc_conf_high,lower_bound_larger_null,task,Features,ccc_segment_mean,ccc_segment_low,ccc_segment_high,concordance_cc-test-agg-average,path,Target,Quality,Task,Model,Survey
0,0.363824,-0.044603,0.590668,False,speechtasks-nilago-happy,eGeMAPS,0.354083,0.027664,0.567943,0.363824,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Denoised,Read-happy,SVR,All
1,0.238113,-0.015007,0.452820,False,speechtasks-sustained_utterance-a,W2V2-LR-LIBRI,0.348391,0.012134,0.539457,0.238113,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Denoised,Sustained /a/,XGBr,All
2,0.350369,0.019678,0.515335,True,speechtasks-nilago-happy,eGeMAPS,0.361475,0.031440,0.514476,0.350369,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Noisy,Read-happy,SVR,All
3,0.340419,0.088999,0.497717,True,speechtasks-nilago-neutral,W2V2-LR-LIBRI,0.288002,0.063689,0.481959,0.340419,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Noisy,Read-neutral,LR,All
4,0.114707,-0.184313,0.315105,False,speechtasks-spontaneous-work_tasks,eGeMAPS,0.082212,-0.099834,0.276044,0.114707,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Denoised,Spontaneous,SVR,All
5,0.172311,-0.122455,0.402716,False,speechtasks-nilago-neutral,W2V2-LR-LIBRI,0.162778,-0.160369,0.336582,0.172311,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Denoised,Read-neutral,LR,All
6,0.151166,-0.113080,0.360953,False,speechtasks-spontaneous-work_tasks,eGeMAPS,0.098678,-0.082959,0.261394,0.151166,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Noisy,Spontaneous,SVR,All
7,0.250923,0.058861,0.402575,True,speechtasks-spontaneous-work_tasks,W2V2-LR-MSP,0.117312,0.020579,0.236165,0.250923,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Noisy,Spontaneous,LR,All
8,0.021402,-0.278219,0.277513,False,speechtasks-sustained_utterance-a,eGeMAPS,0.064470,-0.164220,0.318883,0.021402,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,PHQ-8,Denoised,Sustained /a/,SVR,All
9,0.171450,-0.052023,0.345443,False,speechtasks-nilago-happy,W2V2-LR-LIBRI,0.194951,-0.037036,0.383240,0.171450,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,PHQ-8,Denoised,Read-happy,LR,All


In [10]:
# Define the custom order for "Survey", "Target", and "Quality"
survey_order = ["All", "Daily", "Weekly"]
target_order = ["WHO-5", "PSS-10", "PHQ-8", "Stress-now", "Stress-work"]
quality_order = ["Denoised", "Noisy"]

# Convert the columns to categorical types with the specified order
df_paper["Survey"] = pd.Categorical(
    df_paper["Survey"], categories=survey_order, ordered=True
)
df_paper["Target"] = pd.Categorical(
    df_paper["Target"], categories=target_order, ordered=True
)
df_paper["Quality"] = pd.Categorical(
    df_paper["Quality"], categories=quality_order, ordered=True
)

# Sort the DataFrame by "Survey", "Target", and "Quality"
# Addition: sort also by features (= eGeMAPS vs. wav2vec), since that factor is included now
df_paper = df_paper.sort_values(by=["Survey", "Target", "Features", "Quality"])

In [11]:
df_paper

,ccc_conf_mean,ccc_conf_low,ccc_conf_high,lower_bound_larger_null,task,Features,ccc_segment_mean,ccc_segment_low,ccc_segment_high,concordance_cc-test-agg-average,path,Target,Quality,Task,Model,Survey
1,0.238113,-0.015007,0.452820,False,speechtasks-sustained_utterance-a,W2V2-LR-LIBRI,0.348391,0.012134,0.539457,0.238113,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Denoised,Sustained /a/,XGBr,All
3,0.340419,0.088999,0.497717,True,speechtasks-nilago-neutral,W2V2-LR-LIBRI,0.288002,0.063689,0.481959,0.340419,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Noisy,Read-neutral,LR,All
0,0.363824,-0.044603,0.590668,False,speechtasks-nilago-happy,eGeMAPS,0.354083,0.027664,0.567943,0.363824,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Denoised,Read-happy,SVR,All
2,0.350369,0.019678,0.515335,True,speechtasks-nilago-happy,eGeMAPS,0.361475,0.031440,0.514476,0.350369,/who_5_percentage_score_corrected/raw_0_100-no...,WHO-5,Noisy,Read-happy,SVR,All
5,0.172311,-0.122455,0.402716,False,speechtasks-nilago-neutral,W2V2-LR-LIBRI,0.162778,-0.160369,0.336582,0.172311,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Denoised,Read-neutral,LR,All
7,0.250923,0.058861,0.402575,True,speechtasks-spontaneous-work_tasks,W2V2-LR-MSP,0.117312,0.020579,0.236165,0.250923,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Noisy,Spontaneous,LR,All
4,0.114707,-0.184313,0.315105,False,speechtasks-spontaneous-work_tasks,eGeMAPS,0.082212,-0.099834,0.276044,0.114707,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Denoised,Spontaneous,SVR,All
6,0.151166,-0.113080,0.360953,False,speechtasks-spontaneous-work_tasks,eGeMAPS,0.098678,-0.082959,0.261394,0.151166,/pss_10_total_score/raw_0_40-normalized_0_1/sp...,PSS-10,Noisy,Spontaneous,SVR,All
9,0.171450,-0.052023,0.345443,False,speechtasks-nilago-happy,W2V2-LR-LIBRI,0.194951,-0.037036,0.383240,0.171450,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,PHQ-8,Denoised,Read-happy,LR,All
11,0.207540,-0.010477,0.361362,False,speechtasks-nilago-happy,W2V2-LR-LIBRI,0.212767,-0.002674,0.377122,0.207540,/phq_8_total_score/raw_0_24-normalized_0_1/spe...,PHQ-8,Noisy,Read-happy,LR,All


In [12]:
# Summarize the columns "ccc_conf_mean", "ccc_conf_low", and "ccc_conf_high" into one column "CCC"
# --> these are the session-level values; the segment-level values reside in "ccc_segment_mean" etc.
df_paper["CCC"] = df_paper.apply(
    # Add math mode through `$` for nice display in LaTeX
    lambda row: f"${row['ccc_conf_mean']:.3f} ({row['ccc_conf_low']:.3f} - {row['ccc_conf_high']:.3f})$",
    axis=1,
)

# Add an asterisk to the end of the "CCC" strings in all rows where "lower_bound_larger_null" is True
df_paper.loc[df_paper["lower_bound_larger_null"] == True, "CCC"] += "*"

# Insert the "CCC" column right after the "Quality" column
quality_index = df_paper.columns.get_loc("Quality")
df_paper.insert(quality_index + 1, "CCC", df_paper.pop("CCC"))

In [13]:
# Just keep the columns relevant for the LaTeX output in their designated order
df_paper_full = df_paper.copy()
df_paper = df_paper[["Target", "Quality", "CCC", "Task", "Model", "Features"]]

In [14]:
# The following formatting automations seem not to work reliable (the "Survey")
# column was dropped anyways; so: manually set the best performing model
# in bold face and manially set hlines.

In [15]:
# Function to highlight the best performing model's "CCC" value per "Survey" section
def highlight_best_ccc(df):
    # Create a helper column to extract the numeric part of "CCC"
    df["CCC_float"] = df["CCC"].str.split().str[0].astype(float)

    for survey in df["Survey"].unique():
        survey_df = df[df["Survey"] == survey]
        max_ccc_index = survey_df["CCC_float"].idxmax()
        df.at[max_ccc_index, "CCC"] = f"\\textbf{{{df.at[max_ccc_index, 'CCC']}}}"

    # Drop the helper column
    df = df.drop(columns=["CCC_float"])

    return df


# Highlight the best performing model's "CCC" value per "Survey" section
# df_paper = highlight_best_ccc(df_paper)

In [16]:
# Add horizontal lines at the transitions between "Survey" categories
def add_horizontal_lines(df):
    latex_str = df.to_latex(index=False)
    lines = latex_str.splitlines()
    new_lines = []
    df_index = 0  # Initialize DataFrame row index
    for i, line in enumerate(lines):
        new_lines.append(line)
        # Check if the current line is a data line (not a header or formatting line)
        if (
            i > 3 and i < len(lines) - 1
        ):  # Skip the first three lines (header and \midrule)
            if df_index < len(df) - 1:  # Ensure df_index is within bounds
                prev_survey = df.iloc[df_index]["Survey"]
                curr_survey = df.iloc[df_index + 1]["Survey"]
                if prev_survey != curr_survey:
                    new_lines.append("\\hline")
            df_index += 1  # Increment DataFrame row index only for data lines
    return "\n".join(new_lines)


# Export the DataFrame to LaTeX format with horizontal lines
# latex_code = add_horizontal_lines(df_paper)

In [17]:
df_paper

,Target,Quality,CCC,Task,Model,Features
1,WHO-5,Denoised,$0.238 (-0.015 - 0.453)$,Sustained /a/,XGBr,W2V2-LR-LIBRI
3,WHO-5,Noisy,$0.340 (0.089 - 0.498)$*,Read-neutral,LR,W2V2-LR-LIBRI
0,WHO-5,Denoised,$0.364 (-0.045 - 0.591)$,Read-happy,SVR,eGeMAPS
2,WHO-5,Noisy,$0.350 (0.020 - 0.515)$*,Read-happy,SVR,eGeMAPS
5,PSS-10,Denoised,$0.172 (-0.122 - 0.403)$,Read-neutral,LR,W2V2-LR-LIBRI
7,PSS-10,Noisy,$0.251 (0.059 - 0.403)$*,Spontaneous,LR,W2V2-LR-MSP
4,PSS-10,Denoised,$0.115 (-0.184 - 0.315)$,Spontaneous,SVR,eGeMAPS
6,PSS-10,Noisy,$0.151 (-0.113 - 0.361)$,Spontaneous,SVR,eGeMAPS
9,PHQ-8,Denoised,$0.171 (-0.052 - 0.345)$,Read-happy,LR,W2V2-LR-LIBRI
11,PHQ-8,Noisy,$0.208 (-0.010 - 0.361)$,Read-happy,LR,W2V2-LR-LIBRI


In [18]:
latex_code = df_paper.to_latex(index=False)
print(latex_code)

\begin{tabular}{llllll}
\toprule
Target & Quality & CCC & Task & Model & Features \\
\midrule
WHO-5 & Denoised & $0.238 (-0.015 - 0.453)$ & Sustained /a/ & XGBr & W2V2-LR-LIBRI \\
WHO-5 & Noisy & $0.340 (0.089 - 0.498)$* & Read-neutral & LR & W2V2-LR-LIBRI \\
WHO-5 & Denoised & $0.364 (-0.045 - 0.591)$ & Read-happy & SVR & eGeMAPS \\
WHO-5 & Noisy & $0.350 (0.020 - 0.515)$* & Read-happy & SVR & eGeMAPS \\
PSS-10 & Denoised & $0.172 (-0.122 - 0.403)$ & Read-neutral & LR & W2V2-LR-LIBRI \\
PSS-10 & Noisy & $0.251 (0.059 - 0.403)$* & Spontaneous & LR & W2V2-LR-MSP \\
PSS-10 & Denoised & $0.115 (-0.184 - 0.315)$ & Spontaneous & SVR & eGeMAPS \\
PSS-10 & Noisy & $0.151 (-0.113 - 0.361)$ & Spontaneous & SVR & eGeMAPS \\
PHQ-8 & Denoised & $0.171 (-0.052 - 0.345)$ & Read-happy & LR & W2V2-LR-LIBRI \\
PHQ-8 & Noisy & $0.208 (-0.010 - 0.361)$ & Read-happy & LR & W2V2-LR-LIBRI \\
PHQ-8 & Denoised & $0.021 (-0.278 - 0.278)$ & Sustained /a/ & SVR & eGeMAPS \\
PHQ-8 & Noisy & $0.106 (-0.061 - 0.267

Manually mark the best-performing model with `\mathbf{}` and add vertical lines and spaces for better readability:  
After each feature set bundle (transitioning from wav2vec2 to eGeMAPS): add `\addlinespace[0.25em]`; after each target, add `\hline`.